In [52]:
import requests
import xml.etree.ElementTree as ET

import pandas as pd

In [248]:


def get_users_collection(username):
    collections_endpoint = BASE_URI + 'collection?'
    parameters = f'username={username}'
    return requests.get(collections_endpoint + parameters)

def get_xml_string_from_response(response):
    print(response.text)
    return ET.fromstring(response.text)

def get_game_ids_from_collection(collection_element_tree):
    return [child.attrib['objectid'] for child in collection_element_tree]

def get_games_from_game_ids(game_ids):
    comma_seperated_game_ids = ','.join(game_ids)
    thing_endpoint = BASE_URI + 'thing?'
    parameters = f'id={comma_seperated_game_ids}&stats=1'
    print(thing_endpoint + parameters)
    return requests.get(thing_endpoint + parameters)



In [249]:
class BoardgameXMLParser:
    def __init__(self, board_game_element):
        self.board_game_element = board_game_element
        self.type = self.board_game_element.get('type')
        self.id = self.board_game_element.get('id')
        self.title = self._get_attribute_from_element('name[@type="primary"]', 'value')
        self.description = self.board_game_element.find('description').text
        self.image = self.board_game_element.find('image').text
        self.thumbnail = self.board_game_element.find('thumbnail').text
        self.year_published = self._get_attribute_from_element('yearpublished', 'value')
        self.min_players_from_creators = self._get_attribute_from_element('minplayers', 'value')
        self.max_players_from_creators = self._get_attribute_from_element('maxplayers', 'value')
        self.playing_time = self._get_attribute_from_element('playingtime', 'value')
        self.min_playing_time = self._get_attribute_from_element('minplaytime', 'value')
        self.max_playing_time = self._get_attribute_from_element('maxplaytime', 'value')
        self.min_age = self._get_attribute_from_element('minage', 'value')
        self.average_rating = self._get_attribute_from_element('.//average', 'value')
        self.bayes_average_rating = self._get_attribute_from_element('.//bayesaverage', 'value')
        self.board_game_rank = self._get_attribute_from_element('.//rank[@name="boardgame"]', 'value')  
        
        self.designers = self._get_attributes_from_element('link[@type="boardgamedesigner"]', 'value')
        self.mechanics = self._get_attributes_from_element('link[@type="boardgamemechanic"]', 'value')
        self.categories = self._get_attributes_from_element('link[@type="boardgamecategory"]', 'value')
        
        suggested_player_poll_element, suggested_players_total_votes = self._get_poll_results('poll[@name="suggested_numplayers"]')

        if suggested_players_total_votes == 0:
            self.user_suggested_best_number_of_players = []
            self.user_suggested_recommended_number_of_players = []
        else:   
            suggested_player_counts_df = self._get_suggested_player_counts_dataframe(suggested_player_poll_element)
            suggested_player_counts_df_with_poll_result = self._get_suggested_player_counts_data_with_realtive_amounts(suggested_player_counts_df)
            self.user_suggested_best_number_of_players = self._get_best_player_counts(suggested_player_counts_df_with_poll_result)
            self.user_suggested_recommended_number_of_players = self._get_recommended_player_counts(suggested_player_counts_df_with_poll_result)
        
    def _get_attribute_from_element(self, find_string, attribute_name):
        element = self.board_game_element.find(f'{find_string}')
        return element.get(attribute_name)
    
    def _get_attributes_from_element(self, find_string, attribute_name):
        elements = self.board_game_element.findall(f'{find_string}')
        return [element.get(attribute_name) for element in elements]
    
    def _get_poll_results(self, poll_pattern):
        poll = self.board_game_element.find(poll_pattern)
        return (poll.findall('results'), int(poll.get('totalvotes')))
    
    def _get_suggested_player_counts_dataframe(self, suggested_player_poll_element):
        options = []
        for option in suggested_player_poll_element:
            option_row = {}
            option_row['num_players'] = option.get('numplayers')
            option_row['best'] = int(option.find('result[@value="Best"]').get('numvotes'))
            option_row['recommended'] = int(option.find('result[@value="Recommended"]').get('numvotes'))
            option_row['not_recommended'] = int(option.find('result[@value="Not Recommended"]').get('numvotes'))
            options.append(option_row)
        return pd.DataFrame(options)
    
    def _get_suggested_player_counts_data_with_realtive_amounts(self, suggested_player_counts_df):
        df = suggested_player_counts_df
        df.loc[df['not_recommended'] > df['recommended'] + df['best'], 'poll_result'] = 'not_recommended'
        df.loc[(df['poll_result'] != 'not_recommended') & (df['recommended'] < df['best']), 'poll_result'] = 'best'
        df.loc[(df['poll_result'] != 'not_recommended') & (df['poll_result'] != 'best'), 'poll_result'] = 'recommended'
        return df
        
    def _get_best_player_counts(self, poll_result):
        return poll_result.loc[poll_result['poll_result'] == 'best', 'num_players'].tolist()
    
    def _get_recommended_player_counts(self, poll_result):
        return poll_result.loc[poll_result['poll_result'] != 'not_recommended', 'num_players'].tolist()
    

class CollectionXMLParser():
    def __init__(self, bgg_username):
        pass
    def get_users_collection(username):
        BASE_URI = 'https://www.boardgamegeek.com/xmlapi2/'
        collections_endpoint = BASE_URI + 'collection?'
        parameters = f'username={username}'
        return requests.get(collections_endpoint + parameters)
        

In [250]:
collection = get_users_collection('bobbaganush')

In [251]:
# Uses the api, own cell not to re-send the request

collection_et = get_xml_string_from_response(collection)
game_ids = get_game_ids_from_collection(collection_et)

<?xml version="1.0" encoding="utf-8" standalone="yes"?><items totalitems="101" termsofuse="https://boardgamegeek.com/xmlapi/termsofuse" pubdate="Sun, 09 May 2021 17:55:43 +0000">
		<item objecttype="thing" objectid="68448" subtype="boardgame" collid="39629810">
	<name sortindex="1">7 Wonders</name>
		<yearpublished>2010</yearpublished>			<image>https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__original/img/JQvRoz0xns9LZII74-ygKGDq_Es=/0x0/filters:format(jpeg)/pic860217.jpg</image>
		<thumbnail>https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__thumb/img/ZlG_SRFChObHenw79fAve56_mnk=/fit-in/200x150/filters:strip_icc()/pic860217.jpg</thumbnail>
			<status own="1" prevowned="0" fortrade="0" want="0" wanttoplay="0" wanttobuy="0" wishlist="0"  preordered="0" lastmodified="2017-01-09 07:57:17" />
	<numplays>0</numplays>							</item>
		<item objecttype="thing" objectid="173346" subtype="boardgame" collid="42112237">
	<name sortindex="1">7 Wonders Duel</name>
		<yearpublished>2015</yearpu

In [247]:


games = get_games_from_game_ids(game_ids)
games_et = get_xml_string_from_response(games)


https://www.boardgamegeek.com/xmlapi2/thing?id=68448,173346,31260,13464,205637,230802,170216,174506,174801,224517,171131,822,21385,13,926,325,553,478,178900,198773,39463,225694,104162,36218,5177,283355,157958,72125,246900,175621,177736,199478,169124,175155,37904,31481,23730,291457,193738,227460,198994,154597,859,154203,206051,84159,257501,257,70323,281960,823,143884,463,205059,1927,3943,1621,164928,30549,161936,221107,218603,2651,183006,28143,41114,181,121921,18,237182,438,169786,199727,242277,298638,8222,148228,187645,226840,1897,146508,189035,229853,120677,167791,247030,244522,182028,14996,276894,148951,233078,126163,122328,115746,228051,261594,262906,233867,266192,163602&stats=1


In [238]:
for game in games_et:
    bg = BoardgameXMLParser(game)
    print(bg.title, bg.type, bg.min_players_from_creators, bg.max_players_from_creators, bg.user_suggested_best_number_of_players, bg.user_suggested_best_number_of_players, bg.thumbnail)

7 Wonders boardgame 2 7 ['4', '5'] ['4', '5'] https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__thumb/img/ZlG_SRFChObHenw79fAve56_mnk=/fit-in/200x150/filters:strip_icc()/pic860217.jpg
7 Wonders Duel boardgame 2 2 ['2'] ['2'] https://cf.geekdo-images.com/WzNs1mA_o22ZWTR8fkLP2g__thumb/img/qGf8-LJx4lsNOgpjS6iCs2TyKts=/fit-in/200x150/filters:strip_icc()/pic3376065.jpg
Agricola boardgame 1 5 ['3', '4'] ['3', '4'] https://cf.geekdo-images.com/dDDo2Hexl80ucK1IlqTk-g__thumb/img/GHGdnCfeysoP_34gLnofJcNivW8=/fit-in/200x150/filters:strip_icc()/pic831744.jpg
Alfapet boardgame 2 4 ['2'] ['2'] https://cf.geekdo-images.com/0xIeUho3Eme8PNAgR87nIg__thumb/img/YkgjFAvySdVGmUghJXOWgoB0Shc=/fit-in/200x150/filters:strip_icc()/pic3714347.jpg
Arkham Horror: The Card Game boardgame 1 2 ['2'] ['2'] https://cf.geekdo-images.com/B5F5ulz0UivNgrI9Ky0euA__thumb/img/L8ouPl5jv2Ye9MC4R_Os2zSGigE=/fit-in/200x150/filters:strip_icc()/pic3122349.jpg
Azul boardgame 2 4 ['2'] ['2'] https://cf.geekdo-images.com/tz19PfklMdA

# Här under slutade jag med att få ut alla spel som element trees

In [210]:
print(games.text)

<?xml version="1.0" encoding="utf-8"?><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item type="boardgame" id="68448">
         <thumbnail>https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__thumb/img/ZlG_SRFChObHenw79fAve56_mnk=/fit-in/200x150/filters:strip_icc()/pic860217.jpg</thumbnail>
      <image>https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__original/img/JQvRoz0xns9LZII74-ygKGDq_Es=/0x0/filters:format(jpeg)/pic860217.jpg</image>
                                     				
				<name type="primary" sortindex="1" value="7 Wonders" />
			
						                               				
				<name type="alternate" sortindex="1" value="7 csoda" />
			    				
				<name type="alternate" sortindex="1" value="7 Cudów Świata" />
			    				
				<name type="alternate" sortindex="1" value="7 divů světa" />
			    				
				<name type="alternate" sortindex="1" value="7 чудес" />
			    				
				<name type="alternate" sortindex="1" value="7 สิ่งมหัศจรรย์" />
			    				
				<na

In [17]:
print(first_game.text)

In [17]:
text_results = results.text

In [28]:
text_results

'<?xml version="1.0" encoding="utf-8" standalone="yes"?><items totalitems="101" termsofuse="https://boardgamegeek.com/xmlapi/termsofuse" pubdate="Sun, 09 May 2021 17:55:43 +0000">\n\t\t<item objecttype="thing" objectid="68448" subtype="boardgame" collid="39629810">\n\t<name sortindex="1">7 Wonders</name>\n\t\t<yearpublished>2010</yearpublished>\t\t\t<image>https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__original/img/JQvRoz0xns9LZII74-ygKGDq_Es=/0x0/filters:format(jpeg)/pic860217.jpg</image>\n\t\t<thumbnail>https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__thumb/img/ZlG_SRFChObHenw79fAve56_mnk=/fit-in/200x150/filters:strip_icc()/pic860217.jpg</thumbnail>\n\t\t\t<status own="1" prevowned="0" fortrade="0" want="0" wanttoplay="0" wanttobuy="0" wishlist="0"  preordered="0" lastmodified="2017-01-09 07:57:17" />\n\t<numplays>0</numplays>\t\t\t\t\t\t\t</item>\n\t\t<item objecttype="thing" objectid="173346" subtype="boardgame" collid="42112237">\n\t<name sortindex="1">7 Wonders Duel</na

In [23]:
root = ET.fromstring(text_results)

In [24]:
root.tag

'items'

In [43]:
for child in root:
    print(child.tag, child.attrib)
    for thing in child:
        print(thing.tag, thing.text)

item {'objecttype': 'thing', 'objectid': '68448', 'subtype': 'boardgame', 'collid': '39629810'}
name 7 Wonders
yearpublished 2010
image https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__original/img/JQvRoz0xns9LZII74-ygKGDq_Es=/0x0/filters:format(jpeg)/pic860217.jpg
thumbnail https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__thumb/img/ZlG_SRFChObHenw79fAve56_mnk=/fit-in/200x150/filters:strip_icc()/pic860217.jpg
status None
numplays 0
item {'objecttype': 'thing', 'objectid': '173346', 'subtype': 'boardgame', 'collid': '42112237'}
name 7 Wonders Duel
yearpublished 2015
image https://cf.geekdo-images.com/WzNs1mA_o22ZWTR8fkLP2g__original/img/q_TrwF4VnXgW1dFQgtMJexSXOEA=/0x0/filters:format(jpeg)/pic3376065.jpg
thumbnail https://cf.geekdo-images.com/WzNs1mA_o22ZWTR8fkLP2g__thumb/img/qGf8-LJx4lsNOgpjS6iCs2TyKts=/fit-in/200x150/filters:strip_icc()/pic3376065.jpg
status None
numplays 0
item {'objecttype': 'thing', 'objectid': '31260', 'subtype': 'boardgame', 'collid': '39629792'}
name Agr

In [74]:
game_ids = []
for child in root:
    # print(child.attrib['objectid'])
    game_ids.append(child.attrib['objectid'])
games = ','.join(game_ids[0])

In [75]:
things = requests.get(base_url + 'thing?'  + f'id={games}')

In [76]:
things.text

'<?xml version="1.0" encoding="utf-8"?><items termsofuse="https://boardgamegeek.com/xmlapi/termsofuse"><item type="boardgame" id="6">\n         <thumbnail>https://cf.geekdo-images.com/277POF80AUz2ZE9XSApyDg__thumb/img/FUFTIqOZLoPbHTbD4Zg2uvLry-A=/fit-in/200x150/filters:strip_icc()/pic28424.jpg</thumbnail>\n      <image>https://cf.geekdo-images.com/277POF80AUz2ZE9XSApyDg__original/img/hQCIS_adSN2LfBgUCsEpJ6CsPVQ=/0x0/filters:format(jpeg)/pic28424.jpg</image>\n                                     \t\t\t\t\n\t\t\t\t<name type="primary" sortindex="1" value="Mare Mediterraneum" />\n\t\t\t\n\t\t\t\t\t\t                           \n\t\t\t\t\t\t               \t\t\t\t\t\t\t\t\t\t\t\t\t<description>In the ancient lands along the Mediterranean, players attempt to satisfy their unique victory conditions via trade, war and construction.  This lavishly produced game contains tons of wooden game components and a beautiful roll-out vinyl map.  Players produce a score of different commodities to trade

In [66]:
game_root = ET.fromstring(things.text)

In [78]:
for child in game_root:
    print(child.tag, child.attrib)
    for thing in child:
        print(thing.tag, thing.text, thing.get('name'), thing.get('type'), thing.get('value'))

item {'type': 'boardgame', 'id': '68448'}
thumbnail https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__thumb/img/ZlG_SRFChObHenw79fAve56_mnk=/fit-in/200x150/filters:strip_icc()/pic860217.jpg None None None
image https://cf.geekdo-images.com/RvFVTEpnbb4NM7k0IF8V7A__original/img/JQvRoz0xns9LZII74-ygKGDq_Es=/0x0/filters:format(jpeg)/pic860217.jpg None None None
name None None primary 7 Wonders
name None None alternate 7 csoda
name None None alternate 7 Cudów Świata
name None None alternate 7 divů světa
name None None alternate 7 чудес
name None None alternate 7 สิ่งมหัศจรรย์
name None None alternate 7 원더스
name None None alternate Τα 7 θαύματα του κόσμου
name None None alternate 七大奇蹟
name None None alternate 七大奇迹
name None None alternate 世界の七不思議
description You are the leader of one of the 7 great cities of the Ancient World. Gather resources, develop commercial routes, and affirm your military supremacy. Build your city and erect an architectural wonder which will transcend future times

In [104]:
for child in game_root:
    print(child.tag, child.attrib)
    for thing in child.findall('poll'):
        print(thing.get('name'), thing.get('title'))
        for result in thing.findall('results'):
            print(result.get('numplayers'))
            for res in result.findall('result'):
                print(res.get('value'))
                print(res.get('numvotes'))

item {'type': 'boardgame', 'id': '68448'}
suggested_numplayers User Suggested Number of Players
1
Best
4
Recommended
14
Not Recommended
1233
2
Best
116
Recommended
365
Not Recommended
1042
3
Best
450
Recommended
1081
Not Recommended
187
4
Best
1086
Recommended
708
Not Recommended
38
5
Best
935
Recommended
788
Not Recommended
52
6
Best
426
Recommended
1058
Not Recommended
153
7
Best
360
Recommended
982
Not Recommended
270
7+
Best
30
Recommended
95
Not Recommended
829
suggested_playerage User Suggested Player Age
None
2
0
3
0
4
1
5
0
6
17
8
138
10
187
12
112
14
21
16
3
18
1
21 and up
0
language_dependence Language Dependence
None
No necessary in-game text
301
Some necessary text - easily memorized or small crib sheet
87
Moderate in-game text - needs crib sheet or paste ups
4
Extensive use of text - massive conversion needed to be playable
1
Unplayable in another language
0
item {'type': 'boardgame', 'id': '173346'}
suggested_numplayers User Suggested Number of Players
1
Best
7
Recommende